In [12]:
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer
from pymilvus import MilvusClient
from dotenv import load_dotenv
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"  # 指定使用的GPU设备

load_dotenv()

# 从.env文件加载Milvus连接信息
MILVUS_ENDPOINT = os.getenv("MILVUS_ENDPOINT")
MILVUS_TOKEN = os.getenv("MILVUS_TOKEN")
if not MILVUS_ENDPOINT or not MILVUS_TOKEN:
    raise ValueError("请确保 .env 文件中已设置 MILVUS_ENDPOINT 和 MILVUS_TOKEN")

# 定义常量
CACHE_DIR = "../models"
METADATA_PATH = "./index/corpus_metadata.json"
EMBEDDING_MODEL_NAME = "Qwen3-Embedding-0.6B"
RERANKER_MODEL_NAME = "Qwen3-Reranker-0.6B"
MILVUS_COLLECTION_NAME = "Musique"
TOP_K_RETRIEVAL = 20  # 第一阶段向量检索返回的数量
TOP_K_RERANKED = 10  # 第二阶段重排序后返回的数量

# 加载用于生成查询向量的嵌入模型
embedding_model = SentenceTransformer(
    f"Qwen/{EMBEDDING_MODEL_NAME}",
    cache_folder=f"{CACHE_DIR}/{EMBEDDING_MODEL_NAME}",
    model_kwargs={
        "attn_implementation": "flash_attention_2",
        "torch_dtype": torch.bfloat16,
    },
    tokenizer_kwargs={"padding_side": "left"},
    device="cuda",  # 指定一个主设备
)

# 加载用于重排序的Cross-Encoder模型
reranker_model = CrossEncoder(
    f"tomaarsen/Qwen3-Reranker-0.6B-seq-cls",
    cache_folder=f"{CACHE_DIR}/{RERANKER_MODEL_NAME}-seq-cls",
    model_kwargs={
        "attn_implementation": "flash_attention_2",
        "torch_dtype": torch.bfloat16,
    },
    tokenizer_kwargs={"padding_side": "left",},
    device="cuda",  # 指定一个主设备
)

client = MilvusClient(uri=MILVUS_ENDPOINT, token=MILVUS_TOKEN)

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

In [11]:
res = client.query(
    collection_name=MILVUS_COLLECTION_NAME,
    filter="id like '2hop__47005_159536%'",
    # output_fields=["hop", "title"],
    limit=10,
)

res

data: ["{'hop': 2, 'id': '2hop__47005_159536_0', 'vector': [0.0262451171875, 0.0302734375, -0.00958251953125, 0.04248046875, -0.007659912109375, -0.08251953125, -0.10205078125, -0.0096435546875, 0.022216796875, -0.011962890625, 0.0230712890625, 0.060546875, 0.08642578125, -0.00830078125, -0.03564453125, 0.07861328125, -0.0517578125, 0.0172119140625, 0.046630859375, 0.05322265625, -0.130859375, 0.03662109375, -0.004547119140625, -0.0439453125, -0.00347900390625, 0.01312255859375, -0.04443359375, 0.052490234375, -0.009033203125, 0.01251220703125, -0.05712890625, 0.0830078125, 0.01373291015625, 0.004730224609375, -0.035888671875, -0.01190185546875, 0.029052734375, -0.031494140625, -0.00168609619140625, 0.0380859375, -0.01153564453125, 0.0498046875, -0.03271484375, 0.008544921875, -0.005706787109375, 0.016845703125, 0.0009613037109375, 0.00112152099609375, 0.006744384765625, -0.007354736328125, -0.0311279296875, -0.035400390625, 0.03125, 0.004119873046875, 0.03369140625, 0.0019989013671875

In [13]:
query = ["世界上最高的山峰是什么？", "谁是美国的第一任总统？"]
query_vector = embedding_model.encode_query(
        query, normalize_embeddings=True
    )

query_vector

array([[-0.01220703,  0.04760742, -0.00473022, ...,  0.00180054,
        -0.02770996,  0.00744629],
       [-0.0088501 ,  0.00982666,  0.00156403, ...,  0.02624512,
        -0.04174805,  0.0001545 ]], shape=(2, 1024), dtype=float32)

In [14]:
# 在Milvus中进行搜索
results = client.search(
    collection_name=MILVUS_COLLECTION_NAME,
    data=query_vector,
    anns_field="vector",  # 您在Milvus中定义的向量字段名
    search_params={"metric_type": "IP", "params": {}},
    limit=50,
    output_fields=["text", "title"],  # 输出字段包括文本和ID
)
results

data: [[{'id': '2hop__47005_159536_16', 'distance': 0.7016823291778564, 'entity': {'text': "Chimborazo, 6,267 metres (20,561 ft). Presumed highest from sixteenth century until the beginning of the 19th century. Not in the top 100 highest mountains when measured from sea level, however due to the earth's equatorial bulge this is the farthest point from the Earth's center. Nanda Devi, 7,816 metres (25,643 ft). Presumed highest in the world before Kangchenjunga was sighted in an era when Nepal was still closed to the outside world. Now known to be the 23rd highest mountain in the world. Dhaulagiri, 8,167 metres (26,795 ft). Presumed highest from 1808 until 1847. Now known to be the 7th highest mountain in the world. Kangchenjunga, 8,586 metres (28,169 ft). Presumed highest from 1847 until 1852. Now known to be the 3rd highest mountain in the world. Mount Everest, 8,848 metres (29,029 ft). Established as highest in 1852 and officially confirmed in 1856. K2, 8,611 metres (28,251 ft). Discov

In [15]:
len(results)

2

In [46]:
seen_texts = set()
text = [item["entity"]["text"] for item in results[0] if item["entity"]["title"] not in seen_texts and not seen_texts.add(item["entity"]["title"])]

text

["Chimborazo, 6,267 metres (20,561 ft). Presumed highest from sixteenth century until the beginning of the 19th century. Not in the top 100 highest mountains when measured from sea level, however due to the earth's equatorial bulge this is the farthest point from the Earth's center. Nanda Devi, 7,816 metres (25,643 ft). Presumed highest in the world before Kangchenjunga was sighted in an era when Nepal was still closed to the outside world. Now known to be the 23rd highest mountain in the world. Dhaulagiri, 8,167 metres (26,795 ft). Presumed highest from 1808 until 1847. Now known to be the 7th highest mountain in the world. Kangchenjunga, 8,586 metres (28,169 ft). Presumed highest from 1847 until 1852. Now known to be the 3rd highest mountain in the world. Mount Everest, 8,848 metres (29,029 ft). Established as highest in 1852 and officially confirmed in 1856. K2, 8,611 metres (28,251 ft). Discovered in 1856 before Mt. Everest was officially confirmed, K2's elevation became something 

In [50]:
def format_queries(query, instruction=None):
    prefix = '<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. \
        Note that the answer can only be "yes" or "no".<|im_end|>\n<|im_start|>user\n'
    if instruction is None:
        instruction = (
            "Given a web search query, retrieve relevant passages that answer the query"
        )
    return f"{prefix}<Instruct>: {instruction}\n<Query>: {query}\n"


def format_document(document):
    suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
    return f"<Document>: {document}{suffix}"


task = "Given a web search query, retrieve relevant passages that answer the query"

query = format_queries(query, task)
documents = [format_document(doc) for doc in text]

ranks = reranker_model.rank(
    query=query, documents=text, convert_to_tensor=True, return_documents=True
)

ranks

[{'corpus_id': 4,
  'score': tensor(0.6836, device='cuda:0', dtype=torch.bfloat16),
  'text': 'Tongshanjiabu is a mountain in the Himalayas. At tall, Tongshanjiabu is the 103rd tallest mountain in the world. It sits in the disputed border territory between Bhutan and China. Tongshanjiabu has never been officially climbed.'},
 {'corpus_id': 1,
  'score': tensor(0.6602, device='cuda:0', dtype=torch.bfloat16),
  'text': "Mount Everest, known in Nepali as Sagarmatha (सगरमाथा) and in Tibetan as Chomolungma (ཇོ ་ མོ ་ གླང ་ མ), is Earth's highest mountain above sea level, located in the Mahalangur Himal sub-range of the Himalayas. The international border between Nepal (Province No. 1) and China (Tibet Autonomous Region) runs across its summit point."},
 {'corpus_id': 6,
  'score': tensor(0.6523, device='cuda:0', dtype=torch.bfloat16),
  'text': 'The Matterhorn (German: Matterhorn, (ˈmatərˌhɔrn); Italian: Cervino, (ˈtʃerˈviːno); French: Le Cervin, (mɔ̃ sɛʁvɛ̃)) is a mountain of the Alps, str

In [48]:
ranks = reranker_model.rank(
    query=query, documents=documents, convert_to_tensor=True, return_documents=True
)
ranks

[{'corpus_id': 0,
  'score': tensor(0.9922, device='cuda:0', dtype=torch.bfloat16),
  'text': "<Document>: Chimborazo, 6,267 metres (20,561 ft). Presumed highest from sixteenth century until the beginning of the 19th century. Not in the top 100 highest mountains when measured from sea level, however due to the earth's equatorial bulge this is the farthest point from the Earth's center. Nanda Devi, 7,816 metres (25,643 ft). Presumed highest in the world before Kangchenjunga was sighted in an era when Nepal was still closed to the outside world. Now known to be the 23rd highest mountain in the world. Dhaulagiri, 8,167 metres (26,795 ft). Presumed highest from 1808 until 1847. Now known to be the 7th highest mountain in the world. Kangchenjunga, 8,586 metres (28,169 ft). Presumed highest from 1847 until 1852. Now known to be the 3rd highest mountain in the world. Mount Everest, 8,848 metres (29,029 ft). Established as highest in 1852 and officially confirmed in 1856. K2, 8,611 metres (28,

In [58]:
result = client.query(
        collection_name=MILVUS_COLLECTION_NAME,
        ids="2hop__47005_159536_16",
        output_fields=["text", "title"],
    )
result

data: ['{\'text\': "Chimborazo, 6,267 metres (20,561 ft). Presumed highest from sixteenth century until the beginning of the 19th century. Not in the top 100 highest mountains when measured from sea level, however due to the earth\'s equatorial bulge this is the farthest point from the Earth\'s center. Nanda Devi, 7,816 metres (25,643 ft). Presumed highest in the world before Kangchenjunga was sighted in an era when Nepal was still closed to the outside world. Now known to be the 23rd highest mountain in the world. Dhaulagiri, 8,167 metres (26,795 ft). Presumed highest from 1808 until 1847. Now known to be the 7th highest mountain in the world. Kangchenjunga, 8,586 metres (28,169 ft). Presumed highest from 1847 until 1852. Now known to be the 3rd highest mountain in the world. Mount Everest, 8,848 metres (29,029 ft). Established as highest in 1852 and officially confirmed in 1856. K2, 8,611 metres (28,251 ft). Discovered in 1856 before Mt. Everest was officially confirmed, K2\'s elevat

In [61]:
result[0]

{'text': "Chimborazo, 6,267 metres (20,561 ft). Presumed highest from sixteenth century until the beginning of the 19th century. Not in the top 100 highest mountains when measured from sea level, however due to the earth's equatorial bulge this is the farthest point from the Earth's center. Nanda Devi, 7,816 metres (25,643 ft). Presumed highest in the world before Kangchenjunga was sighted in an era when Nepal was still closed to the outside world. Now known to be the 23rd highest mountain in the world. Dhaulagiri, 8,167 metres (26,795 ft). Presumed highest from 1808 until 1847. Now known to be the 7th highest mountain in the world. Kangchenjunga, 8,586 metres (28,169 ft). Presumed highest from 1847 until 1852. Now known to be the 3rd highest mountain in the world. Mount Everest, 8,848 metres (29,029 ft). Established as highest in 1852 and officially confirmed in 1856. K2, 8,611 metres (28,251 ft). Discovered in 1856 before Mt. Everest was officially confirmed, K2's elevation became so